In [1]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.cross_validation import StratifiedShuffleSplit

/home/neo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
LOG_DIR = 'text_logs'
metadata = os.path.join(LOG_DIR,'metadata.tsv')
labels = pickle.load(open("./features/labels.pkl", "rb"))
features = np.array(pickle.load(open("./features/features.pkl", "rb")))
words = pickle.load(open("./features/final_words.pkl","rb"))
features = [np.ndarray.tolist(element) for element in features] 

In [3]:
print("Length of labels:{0}\nLength of features:{1}\nLength of words:{2}\n"\
      .format(len(labels),len(features),len(words)))

Length of labels:11764
Length of features:11764
Length of words:11764



In [4]:
# labels = [label[0] for label in labels]
new_labels = []
for label in labels:
    new_labels.append(label[0])

In [5]:
# split = min(labels.count("cooking"),labels.count("robotics"))
# strat = ["cooking" for x in range(split//2)] + ["robotics" for x in range(split//2)] 

In [6]:
# stratSplit = StratifiedShuffleSplit(labels, 1, test_size=0.77,random_state=42)
# StratifiedShuffleSplit(y, n_iter=1, test_size=0.5)

In [7]:
feature_train, feature_test, label_train, label_test = train_test_split(features, new_labels, \
                                                    test_size=0.5, random_state=0)
# word_train, word_test, label_train, label_test = train_test_split(words, new_labels, \
#                                                     test_size=0.5, random_state=0)

In [8]:
len(feature_train)

5882

In [9]:
# mnist = input_data.read_data_sets('MNIST_data')
X_init = tf.placeholder(tf.float32, shape=(5882, 100))
images = tf.Variable(X_init)
# images = tf.Variable(feature_train, name='features')
# images = tf.Variable(X_test, name='images')
# im_labels = []
# for word in y_test:
#     im_labels.append(labels[words.index(word)])

In [10]:
with open(metadata, 'w') as metadata_file:
    for row in label_train:
        metadata_file.write('%s\n' % row)

In [11]:
with tf.Session() as sess:
    saver = tf.train.Saver([images])

    sess.run(tf.initialize_all_variables(), feed_dict={X_init: feature_train})
    saver.save(sess, os.path.join(LOG_DIR, 'features.ckpt'))

    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = images.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = metadata
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
